# Benötigte Pakete
bs4 (BeatifulSoup): Extrahieren von Daten aus HTML-Seiten<br>
requests: Hier HTTP-Anfrage an Website<br>
pandas: Hier zum Einlesen und Datenkonvertierung


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Funktion zum Scrapen der aktuell freien Plätze definieren
Funktion bekommt Parameter (Kürzel des Parkhauses) übergeben und überschreibt die Daten in der csv-Datei des entsprechenden Parkhauses. Dabei wird der Inhalt der Datei um den neuen Datenpunkt ergänzt.

In [ ]:
def scrape_free_spots(parking_lot):
    BASE_URL = "https://web1.karlsruhe.de/service/Parken/detail.php?id="

    # Einlesen der bisherigen CSV des Parkhauses
    parking_data = pd.read_csv(f"./data/{parking_lot}.csv", index_col=0)
    # timestamp als "datetime"-Format setzen
    parking_data["timestamp"] = pd.to_datetime(parking_data["timestamp"])

    try:
        # Zugriff auf Website
        res = requests.get(f"{BASE_URL}{parking_lot}")
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")

            # Auslesen der aktuell freien Plätze anhand nächsten Element nach Element mit textContent "Gesamtzahl der freien Stellplätze"
            free_spots = soup.find(
                string="Gesamtzahl der freien Stellplätze"
            ).find_next("td")

            # Erstes h1 (Parkhaustitel) finden (als Ausgangspunkt, um zu timestamp zu kommen)
            title = soup.find("h1")
            time_string = title.find_next("p").find_next("p")

            # Datetime aus Zeitpunktbeschreibung extrahieren
            time_string = time_string.text.split("von")[1].split("Uhr")[0]
            time_string = time_string.split("-")
            time_string = time_string[0].strip() + " " + time_string[1].strip()
            timestamp = pd.to_datetime(time_string, format="%d.%m.%Y %H:%M")

            # Neuen Datenpunkt in parking_data ergänzen, wenn nicht schon vorhanden
            if timestamp not in parking_data["timestamp"].values:
                parking_data.loc[len(parking_data)] = [timestamp, free_spots.text]
                print("Data added: ", timestamp)
                print(f"   {free_spots.text} free spots")
                parking_data.to_csv(f"./data/{parking_lot}.csv")
            else:
                print("Data already exists: ", timestamp)

        else:
            print("Error: ", res.status_code)
    except Exception as e:
        print("Error: ", e)

# Scraper ausführen für alle Parkhäuser
(Hier wurde **if \_\_name\_\_ == '\_\_main\_\_'** entfernt, da Jupyter Notebook)

In [ ]:
# Definition Parkhauskuerzel für ID in URL
parking_lots = [
    "K01",
    "K02",
    "K03",
    "K04",
    "N02",
    "N03",
    "N05",
    "N06",
    "N07",
    "S01",
    "S02",
    "S03",
    "S04",
    "S05",
    "S06",
    "S07",
    "W01",
    "W02",
    "W03",
    "W04",
]

for parking_lot in parking_lots:
    scrape_free_spots(parking_lot)